In [ ]:
import pandas as pd
import numpy as np
from plprob.utils import (split_actuals_hist_future, split_forecasts_hist_future)
from plprob.predictor import PlPredictor
import matplotlib.pyplot as plt
from math import ceil
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

# Load data with forecast issued at 23:00

In [ ]:
rto_actual_df = pd.read_csv('../data/PJM/RTO_ACT.csv', parse_dates=['Time'], index_col='Time')
rto_forecast_df = pd.read_csv('../data/PJM/RTO_FCST.csv', parse_dates=['Issue_time', 'Forecast_time'])

### Remove weekends and holidays

In [ ]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
business_days = pd.bdate_range(start='2011-01-01',end='2023-12-31', freq=us_bd, tz='US/Eastern')

rto_actual_df = rto_actual_df.loc[rto_actual_df.index.floor('D').isin(business_days)]
rto_forecast_df = rto_forecast_df[rto_forecast_df['Forecast_time'].dt.floor('D').isin(business_days)]

### Run simulations from 2012 to 2022

In [ ]:
hist_peak = sorted(rto_actual_df[rto_actual_df.index<='2012-01-01'].sort_values('RTO', ascending=False).values[0:5].ravel().tolist())[0]
alerts = dict()
thres = 0.8
num_of_cps = 5

for y in range(2012, 2023):
    
    cp_probs = dict()
    daily_peaks = dict()
    this_year_cps = []

    start = str(y) + '-06-01'
    end = str(y) + '-09-30'
        
    for day in pd.bdate_range(start=start,end=end, freq=us_bd, tz='US/Eastern'):
        
        start_date = day.strftime('%Y-%m-%d')
        print(start_date)
        
        start_time = pd.to_datetime(start_date).tz_localize('US/Eastern')
        timesteps = pd.date_range(start=start_time, periods=24, freq='H')
    
        # Separate historical and future data
        (load_actual_hists,
             load_actual_futures) = split_actuals_hist_future(
                    rto_actual_df, timesteps)
        
        (load_forecast_hists,
             load_forecast_futures) = split_forecasts_hist_future(
                    rto_forecast_df, timesteps)
    
        # Fit model and compute probability
        predictor = PlPredictor(load_actual_hists, load_forecast_hists, start_time, 
                                num_of_cps, this_year_cps, forecast_lead_time_in_hour=1)
        predictor.fit(0.05, 0.05)
    
        predictor.create_scenario(1000, load_forecast_futures)
        predictor.compute_cp_probs()
    
        # Update historical CPs
        today_peak = load_actual_futures.loc[timesteps, 'RTO'].max()
        predictor.update_cp(today_peak)
        if this_year_cps != predictor.hist_cps:
            this_year_cps = predictor.hist_cps

        if load_forecast_futures[load_forecast_futures['Forecast_time'].isin(timesteps)]['RTO'].max() > thres * hist_peak:
            alerts[start_date] = predictor.cp_prob
    
    hist_peak = min(hist_peak, sorted(this_year_cps)[0])

In [ ]:
alert_days = set()
for d, prob in alerts.items():
    k = 4
    while k >= 0:
        if k in prob and prob[k] > 0.6:
            alert_days.add(d)
            break
        else:
            k -= 1

In [ ]:
true_cps = set()
for y in range(2012, 2023):
    cps = rto_actual_df[rto_actual_df.index.year==y].sort_values('RTO', ascending=False).index.floor('D').unique()[0:5]
    for cp in cps:
        true_cps.add(cp.strftime('%Y-%m-%d'))

### Capture all CPs?

In [ ]:
true_cps.issubset(alert_days)

In [ ]:
print(f"missing CP days are {true_cps.difference(alert_days)}")
print(f"number of alerts is {len(alert_days)}")
print(f"number of false alerts is {len(alert_days) - len(true_cps) + len(true_cps.difference(alert_days))}")

# Load data with forecast issued at 11:00

In [ ]:
rto_actual_df = pd.read_csv('../data/PJM/RTO_ACT.csv', parse_dates=['Time'], index_col='Time')
rto_forecast_df = pd.read_csv('../data/PJM/RTO_FCST_HORI_12.csv', parse_dates=['Issue_time', 'Forecast_time'])

### Remove weekends and holidays

In [ ]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
business_days = pd.bdate_range(start='2011-01-01',end='2023-12-31', freq=us_bd, tz='US/Eastern')

rto_actual_df = rto_actual_df.loc[rto_actual_df.index.floor('D').isin(business_days)]
rto_forecast_df = rto_forecast_df[rto_forecast_df['Forecast_time'].dt.floor('D').isin(business_days)]

### Run simulations from 2012 to 2022

In [ ]:
hist_peak = sorted(rto_actual_df[rto_actual_df.index<='2012-01-01'].sort_values('RTO', ascending=False).values[0:5].ravel().tolist())[0]
alerts = dict()
thres = 0.8
num_of_cps = 5

for y in range(2012, 2023):
    
    cp_probs = dict()
    daily_peaks = dict()
    this_year_cps = []

    start = str(y) + '-06-01'
    end = str(y) + '-09-30'
        
    for day in pd.bdate_range(start=start,end=end, freq=us_bd, tz='US/Eastern'):
        
        start_date = day.strftime('%Y-%m-%d')
        print(start_date)

        if start_date == '2022-08-05':
            # Forecast data missing on 2022-08-05
            continue
        else:
            start_time = pd.to_datetime(start_date).tz_localize('US/Eastern') + pd.Timedelta(12, unit='H')
            timesteps = pd.date_range(start=start_time, periods=12, freq='H')
        
            # Separate historical and future data
            (load_actual_hists,
                 load_actual_futures) = split_actuals_hist_future(
                        rto_actual_df, timesteps)
            
            (load_forecast_hists,
                 load_forecast_futures) = split_forecasts_hist_future(
                        rto_forecast_df, timesteps)
        
            # Fit model and compute probability
            predictor = PlPredictor(load_actual_hists, load_forecast_hists, start_time, 
                                    num_of_cps, this_year_cps, num_of_horizons=12, forecast_lead_time_in_hour=1)
            predictor.fit(0.05, 0.05)
        
            predictor.create_scenario(1000, load_forecast_futures)
            predictor.compute_cp_probs()
        
            # Update historical CPs
            today_peak = load_actual_futures.loc[timesteps, 'RTO'].max()
            predictor.update_cp(today_peak)
            if this_year_cps != predictor.hist_cps:
                this_year_cps = predictor.hist_cps
    
            if load_forecast_futures[load_forecast_futures['Forecast_time'].isin(timesteps)]['RTO'].max() > thres * hist_peak:
                alerts[start_date] = predictor.cp_prob
    
    hist_peak = min(hist_peak, sorted(this_year_cps)[0])

In [ ]:
alert_days = set()
for d, prob in alerts.items():
    k = 4
    while k >= 0:
        if k in prob and prob[k] > 0.6:
            alert_days.add(d)
            break
        else:
            k -= 1

In [ ]:
true_cps = set()
for y in range(2012, 2023):
    cps = rto_actual_df[rto_actual_df.index.year==y].sort_values('RTO', ascending=False).index.floor('D').unique()[0:5]
    for cp in cps:
        true_cps.add(cp.strftime('%Y-%m-%d'))

### Capture all CPs?

In [ ]:
true_cps.issubset(alert_days)

In [ ]:
print(f"missing CP days are {true_cps.difference(alert_days)}")
print(f"number of alerts is {len(alert_days)}")
print(f"number of false alerts is {len(alert_days) - len(true_cps) + len(true_cps.difference(alert_days))}")